<a href="https://colab.research.google.com/github/olanigan/yoruba_asr/blob/main/W2V2_Bert_Yor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install datasets torchaudio jiwer -q
!pip install --upgrade transformers -q
!pip install accelerate -U -q


In [2]:
from huggingface_hub import notebook_login

notebook_login()


# Create Wav2Vec2CTCTokenizer

In [3]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_16_0", "yo", split="train+validation", use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_16_0", "yo", split="test", use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_16_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1212it [00:00, 33312.78it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 863it [00:00, 35518.44it/s]


...


Reading metadata...: 998it [00:00, 51296.10it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1169it [00:00, 63080.76it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 229it [00:00, 40326.46it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_16_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])


In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)


,sentence,variant
0,Oluwapọ̀nmile sọ ìrírí rẹ̀ lọ́wọ́ àwọn sójà tó fìyàjẹ́ ní Ibadan.,
1,Kúnlé àti Afọláyan gbéná wojú ara wọn lórí ayélujára.,
2,Ìró ìbọn yìí dún gbàù lásìkò ìpàdé àwọ ọmọlẹ́yìn Arẹ́gbẹ́ṣọlá,
3,Ọkùnrin méjì ló fi mágùn sí orí ìyàwó rẹ̀ ní Bashọ̀run,
4,Oríṣiríṣi ouńjẹ ni ó wà fún àwọn àlejò ní ọjọ́ ọdún.,
5,Ọ̀fẹ́ ni ẹ o máa gbọ́ ìròyìn Yorùbá lórí àwọn ẹ̀rọ ayélujára.,
6,Ọ̀gá ọlọ́pàá pàṣẹ pé kí wọ́n ṣé ìwádìí lórí ọlọ́pàá inú àwòrán.,
7,Wọ́n fẹ́ dá ère tí wọ́n jí padà sí ilé gbọ̀ngàn àṣà ìbílẹ̀.,
8,"Kò sí èyí tí ò ṣe pàtàkì nínú ọmọ, yálà obìnrin tàbí ọkùnrin.",
9,Àwọn darandaran kéde iye mílíọ̀nù tí wọ́n fẹ́ máa ta màálù kan,


In [6]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\»\«]'

def remove_special_characters(batch):
    # remove special characters
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()

    return batch

common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


Map:   0%|          | 0/2075 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [7]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))


,sentence,variant
0,ǹjẹ́ o mọ̀ nípa bídèmí tó wọ gàu ní amẹ́ríkà lórí ẹ̀sùn jì̀bì̀tì̀,
1,ìyàwó yìí máa ń gbá ọkọ rẹ̀ létí lẹ́ẹ̀kọ̀kan,
2,abíọ́dún kìí jẹ tàbí fojú rí àlùbọ́sà rárá nínú óuńjẹ,
3,bọ́lá tinúbú kò rán ẹnikẹ́ni níṣẹ́ láti bá òun polongo ìbò,
4,tí a bá kó ìyàwó jọ láì tọ́jú wọn ọlọ́run á bínú síi,
5,àwọn ọ̀dọ́ lé kábíyèsí jáde láàfin,
6,àjọ elépo rọ̀bì ní kó sáyé fífi owó kún owó epo bẹntiró,
7,àwọn agbófinró ti bẹ̀rẹ̀ ètò ìdánimọ̀ fún àwọn ọlọ́pàá,
8,adájọ́ ti pè wá láti jẹ́rìí sí ìgbẹ́jọ́ bàbá ìjẹ̀shà ní ọ̀túnla,
9,ọmọba fẹ́mi oyéwùnmí tí ọ̀pọ̀ mọ̀ sí ládití ti jáde láyé,


In [8]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/2075 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [9]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


{' ': 0,
 'a': 1,
 'b': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'w': 20,
 'y': 21,
 'à': 22,
 'á': 23,
 'è': 24,
 'é': 25,
 'ì': 26,
 'í': 27,
 'ò': 28,
 'ó': 29,
 'ù': 30,
 'ú': 31,
 'ń': 32,
 'ǹ': 33,
 '̀': 34,
 '́': 35,
 '̄': 36,
 '̣': 37,
 'ṣ': 38,
 'ẹ': 39,
 'ọ': 40,
 'ụ': 41,
 '’': 42}

In [10]:
# prompt: Display redundant characters in vocab_dict

# Create a set of all unique characters in the vocabulary
unique_chars = set(vocab_dict.keys())

# Create a set of redundant characters
redundant_chars = set()

# Iterate through the vocabulary and find characters that appear more than once
for char in unique_chars:
  count = 0
  for key, value in vocab_dict.items():
    if char == key:
      count += 1
  if count > 1:
    redundant_chars.add(char)

# Display the redundant characters
print("Redundant characters in vocab_dict:")
for char in redundant_chars:
  print(char)


Redundant characters in vocab_dict:


In [11]:
%%writefile cleanup.py3
def remove_latin_characters(batch):
    batch["sentence"] = re.sub(r'[a-z]+', '', batch["sentence"])
    return batch

# remove latin characters
common_voice_train = common_voice_train.map(remove_latin_characters)
common_voice_test = common_voice_test.map(remove_latin_characters)

# extract unique characters again
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict


Writing cleanup.py3


In [12]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
len(vocab_dict)


43

In [13]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)


45

In [14]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)


In [15]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


## HF Tokenizer push

In [21]:
print(tokenizer)

Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=45, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	43: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	44: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	45: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	46: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [22]:
if isinstance(tokenizer, Wav2Vec2CTCTokenizer):
    print("Tokenizer is a Wav2Vec2CTCTokenizer")
else:
    print("Tokenizer is not a Wav2Vec2CTCTokenizer")

Tokenizer is a Wav2Vec2CTCTokenizer


In [23]:
import re
repo_name = "w2v-bert-yoruba-CV16"


In [24]:
!huggingface-cli repo view $repo_name

usage: huggingface-cli <command> [<args>] repo [-h] {create} ...
huggingface-cli <command> [<args>] repo: error: argument {create}: invalid choice: 'view' (choose from 'create')


In [25]:

tokenizer.push_to_hub(repo_name)


CommitInfo(commit_url='https://huggingface.co/olanigan/w2v-bert-yoruba-CV16/commit/a7c8a9e86e7a7defc07d7ee1e2b737455f0825ae', commit_message='Upload tokenizer', commit_description='', oid='a7c8a9e86e7a7defc07d7ee1e2b737455f0825ae', pr_url=None, pr_revision=None, pr_num=None)

# Create SeamlessM4TFeatureExtractor

In [26]:
# Import Feature Extractor
from transformers import SeamlessM4TFeatureExtractor

feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

In [27]:
print(repo_name)

w2v-bert-yoruba-CV16


In [28]:
from transformers import Wav2Vec2BertProcessor
# repo_name = "olanigan/w2v-bert-yoruba-CV16"
processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.push_to_hub(repo_name)


CommitInfo(commit_url='https://huggingface.co/olanigan/w2v-bert-yoruba-CV16/commit/3e703e3b81677aeb8e043d887a68ecbb2503c968', commit_message='Upload processor', commit_description='', oid='3e703e3b81677aeb8e043d887a68ecbb2503c968', pr_url=None, pr_revision=None, pr_num=None)

## Preprocess Data

### Verify Audio data

In [29]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/52719897f0665044c22e829b0731227f4cead9dffddf7a387034bd6106d01657/yo_train_0/common_voice_yo_36840058.mp3'

In [30]:
common_voice_train[0]["audio"]


{'path': '/root/.cache/huggingface/datasets/downloads/extracted/52719897f0665044c22e829b0731227f4cead9dffddf7a387034bd6106d01657/yo_train_0/common_voice_yo_36840058.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.13843998e-09, 4.96209518e-09, 2.56767763e-09]),
 'sampling_rate': 48000}

In [31]:
# Set Audio feature to match 16K sampling rate of Wav2Vec2-BERT

common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))


In [32]:
common_voice_train[0]["audio"]


{'path': '/root/.cache/huggingface/datasets/downloads/extracted/52719897f0665044c22e829b0731227f4cead9dffddf7a387034bd6106d01657/yo_train_0/common_voice_yo_36840058.mp3',
 'array': array([ 1.04083409e-17,  1.73472348e-18,  3.46944695e-18, ...,
        -4.20848156e-09,  2.02898764e-09,  6.15687945e-09]),
 'sampling_rate': 16000}

In [34]:
# Verify Audio

import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


èdè kan àti ẹ̀sìn kan ló lè mú ìṣọ̀kan bá orílẹ̀ èdè nàìjíríà


In [35]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])


Target text: ẹbí ọmọ tí wọ́n rí òkú rẹ̀ ní ikú rẹ̀ kìí ṣe ojú lásán
Input array shape: (90432,)
Sampling rate: 16000


### Preprocess data for Training

In [36]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["input_length"] = len(batch["input_features"])

    batch["labels"] = processor(text=batch["sentence"]).input_ids
    return batch
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/2075 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

# Training


## Trainer setup

In [37]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2BertProcessor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )
        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [38]:
# Define evaluation metric as WER (Word Error Count)
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


<ipython-input-38-10bf048bde63>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


## Load Checkpoint

In [39]:
from transformers import Wav2Vec2BertForCTC

model = Wav2Vec2BertForCTC.from_pretrained(
    "facebook/w2v-bert-2.0",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    add_adapter=True,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

Some weights of Wav2Vec2BertForCTC were not initialized from the model checkpoint at facebook/w2v-bert-2.0 and are newly initialized: ['adapter.layers.0.ffn.intermediate_dense.bias', 'adapter.layers.0.ffn.intermediate_dense.weight', 'adapter.layers.0.ffn.output_dense.bias', 'adapter.layers.0.ffn.output_dense.weight', 'adapter.layers.0.ffn_layer_norm.bias', 'adapter.layers.0.ffn_layer_norm.weight', 'adapter.layers.0.residual_conv.bias', 'adapter.layers.0.residual_conv.weight', 'adapter.layers.0.residual_layer_norm.bias', 'adapter.layers.0.residual_layer_norm.weight', 'adapter.layers.0.self_attn.linear_k.bias', 'adapter.layers.0.self_attn.linear_k.weight', 'adapter.layers.0.self_attn.linear_out.bias', 'adapter.layers.0.self_attn.linear_out.weight', 'adapter.layers.0.self_attn.linear_q.bias', 'adapter.layers.0.self_attn.linear_q.weight', 'adapter.layers.0.self_attn.linear_v.bias', 'adapter.layers.0.self_attn.linear_v.weight', 'adapter.layers.0.self_attn_conv.bias', 'adapter.layers.0.self_

## Train model

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=600,
  eval_steps=300,
  logging_steps=300,
  learning_rate=5e-5,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)


In [41]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
300,2.177100,0.922670,0.730194
600,0.560000,0.836052,0.647440


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=650, training_loss=1.2947419885488656, metrics={'train_runtime': 4290.7212, 'train_samples_per_second': 4.836, 'train_steps_per_second': 0.151, 'total_flos': 3.5451611731498895e+18, 'train_loss': 1.2947419885488656, 'epoch': 10.0})

In [42]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

events.out.tfevents.1706544594.0b7743d612c9.1404.0:   0%|          | 0.00/6.93k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/olanigan/w2v-bert-yoruba-CV16/commit/f80f566c485dd932cdcf4bcdbb31a828daef8814', commit_message='End of training', commit_description='', oid='f80f566c485dd932cdcf4bcdbb31a828daef8814', pr_url=None, pr_revision=None, pr_num=None)